# [4.7 Solving CSPs using Stochastic Local Search](http://artint.info/2e/html/ArtInt2e.Ch4.S7.html)

- [Implementation Details](http://artint.info/AIPython/aipython.pdf#page=63) (page 63)

## About

For large CSPs, systematically searching through the space of assignments of values to variables will prove unviable. This section considers a method intended to work in these very large spaces, __stochastic local search__. Stochastic local search does not systematically search the whole search space but they are designed to find solutions quickly on average. It does not guarantee that a solution will be found even if one exists, and so it is not able to prove that no solution exists. It is often the method of choice for applications where solutions are known to exist or are very likely to exist.



## Instructions

Each section header contains a link to the corresponding chapter in the accompanying textbook, and an "Implementation Details" link provided throughout tells you how the implementation works. Before using this notebook, make sure you have followed the [installation instructions](https://aispace2.github.io/AISpace2/install.html) beforehand.

You can run each cell by selecting it and pressing *Ctrl+Enter* in Windows or *Shift+Return* in Mac. Alternatively, you can click the *Play* button in the toolbar, to the left of the stop button. 

For more information, including how the code in this notebook differs from that in [AIPython](aipython.org), check out the [Reference](https://aispace2.github.io/AISpace2/reference.html).

In [ ]:
from aispace2.jupyter.csp import Displayable, visualize
from aipython.cspSLS import Updatable_priority_queue
import random

class SLSearcher(Displayable):
    """A search problem directly from the CSP..

    A node is a variable:value dictionary"""
    
    def __init__(self, csp):
        self.csp = csp
        self.variables_to_select = {var for var in self.csp.variables
                                    if len(self.csp.domains[var]) > 1}
        # Create assignment and conflicts set
        self.current_assignment = None  # this will trigger a random restart
        self.number_of_steps = 1  # number of steps after the initialization
        self.max_steps=500
        super().__init__()

    def restart(self):
        """creates a new total assignment and the conflict set
        """
        self.current_assignment = {var: random_sample(dom) for
                                   (var, dom) in self.csp.domains.items()}
        self.display(2, "Initial assignment", self.current_assignment)
        self.conflicts = set()
        for con in self.csp.constraints:
            if not con.holds(self.current_assignment):
                self.conflicts.add(con)
                
        self.display(2, "Conflicts:", self.conflicts)
        self.variable_pq = None

    @visualize
    def search(self, prob_best=1.0, prob_anycon=1.0):
        """
        returns the number of steps or None if these is no solution
        if there is a solution, it can be found in self.current_assignment
        """
        if self.current_assignment is None:
            self.restart()
            self.number_of_steps += 1
            if not self.conflicts:
                return self.number_of_steps
        if prob_best > 0:  # we need to maintain a variable priority queue
            return self.search_with_var_pq(prob_best, prob_anycon)
        else:
            return self.search_with_any_conflict(prob_anycon)

    def search_with_any_conflict(self, prob_anycon=1.0):
        """Searches with the any_conflict heuristic.
        This relies on just maintaining the set of conflicts; 
        it does not maintain a priority queue
        """
        self.variable_pq = None   # we are not maintaining the priority queue.
        # This ensures it is regenerated if needed.
        for i in range(self.max_steps):
            self.number_of_steps += 1
            if random.random() < prob_anycon:
                con = random_sample(self.conflicts)  # pick random conflict
                var = random_sample(con.scope)   # pick variable in conflict
            else:
                var = random_sample(self.variables_to_select)
            if len(self.csp.domains[var]) > 1:
                val = random_sample(self.csp.domains[var] -
                                    {self.current_assignment[var]})
                self.display(2, "Assigning", var, "=", val)
                self.current_assignment[var] = val
                for varcon in self.csp.var_to_const[var]:
                    if varcon.holds(self.current_assignment):
                        if varcon in self.conflicts:
                            self.conflicts.remove(varcon)
                            self.display(3, "Became consistent", varcon)
                        else:
                            self.display(3, "Still consistent", varcon)
                    else:
                        if varcon not in self.conflicts:
                            self.conflicts.add(varcon)
                            self.display(3, "Became inconsistent", varcon)
                        else:
                            self.display(3, "Still inconsistent", varcon)
                self.display(2, "Conflicts:", self.conflicts)
            if not self.conflicts:
                self.display(1, "Solution found", self.current_assignment,
                             "in", self.number_of_steps, "steps")
                return self.number_of_steps
        self.display(1, "No solution in", self.number_of_steps, "steps",
                     len(self.conflicts), "conflicts remain")
        return None

    def search_with_var_pq(self, prob_best=1.0, prob_anycon=1.0):
        """search with a priority queue of variables.
        This is used to select a variable with the most conflicts.
        """
        if not self.variable_pq:
            self.create_pq()
        pick_best_or_con = prob_best + prob_anycon
        for i in range(self.max_steps):
            self.number_of_steps += 1
            randnum = random.random()
            # Pick a variable
            if randnum < prob_best:  # pick best variable
                var, oldval = self.variable_pq.top()
            elif randnum < pick_best_or_con:  # pick a variable in a conflict
                con = random_sample(self.conflicts)
                var = random_sample(con.scope)
            else:  # pick any variable that can be selected
                var = random_sample(self.variables_to_select)
            if len(self.csp.domains[var]) > 1:   # var has other values
                # Pick a value
                val = random_sample(self.csp.domains[var] -
                                    {self.current_assignment[var]})
                self.display(2, "Assigning", var, "=", val)
                # Update the priority queue
                var_differential = {}
                self.current_assignment[var] = val
                for varcon in self.csp.var_to_const[var]:
                    self.display(3, "Checking", varcon)
                    if varcon.holds(self.current_assignment):
                        if varcon in self.conflicts:  # was incons, now consis
                            self.display(3, "Became consistent", varcon)
                            self.conflicts.remove(varcon)
                            for v in varcon.scope:  # v is in one fewer conflicts
                                var_differential[v] = var_differential.get(
                                    v, 0) - 1
                        else:
                            self.display(3, "Still consistent", varcon)
                    else:
                        if varcon not in self.conflicts:  # was consis, not now
                            self.display(3, "Became inconsistent", varcon)
                            self.conflicts.add(varcon)
                            for v in varcon.scope:  # v is in one more conflicts
                                var_differential[v] = var_differential.get(
                                    v, 0) + 1
                        else:
                            self.display(3, "Still inconsistent", varcon)
                self.variable_pq.update_each_priority(var_differential)
                self.display(2, "Conflicts:", self.conflicts)
            if not self.conflicts:  # no conflicts, so solution found
                self.display(1, "Solution found", self.current_assignment, "in",
                             self.number_of_steps, "steps")
                return self.number_of_steps
        self.display(1, "No solution in", self.number_of_steps, "steps",
                     len(self.conflicts), "conflicts remain")
        return None

    def create_pq(self):
        """Create the variable to number-of-conflicts priority queue.
        This is needed to select the variable in the most conflicts.

        The value of a variable in the priority queue is the negative of the
        number of conflicts the variable appears in.
        """
        self.variable_pq = Updatable_priority_queue()
        var_to_number_conflicts = {}
        for con in self.conflicts:
            for var in con.scope:
                var_to_number_conflicts[var] = var_to_number_conflicts.get(
                    var, 0) + 1
        for var, num in var_to_number_conflicts.items():
            if num > 0:
                self.variable_pq.add(var, -num)
                
def random_sample(st):
    """selects a random element from set st"""
    return random.sample(st,1)[0]

In [ ]:
from aipython.cspExamples import simple_csp1, simple_csp2, extended_csp, crossword1, crossword2, crossword2d
from IPython.display import display

se0 = SLSearcher(csp=simple_csp1)
se0.sleep_time = 0.2 # The time delay between consecutive step in autosolve.
se0.detail_level = 1 # 0=no text, 1=truncated text, 2=full text
se0.text_size = 15
se0.max_steps = 500

se0.search(prob_best=0, prob_anycon=1.0)
"""
prob_best is the probability of selecting a best variable (one involving the most conflicts)
prob_anycon is the probability that the any-conflict strategy is used,
    which selects a variable at random that is in a conflict, 
    assuming that it is not picking a best variable
"""
display(se0)

# Plotting Runtime Distributions
- [Implementation Details](http://artint.info/AIPython/aipython.pdf#page=69) (page 69)

Due to the stochastic nature of local search, multiple runs over the same problem will yield different results. Instead, we look at the runtime distribution - the number of runs that find a solution within a given number of steps.

In [ ]:
from aipython.cspSLS import Runtime_distribution
from aipython.cspExamples import simple_csp1, simple_csp2, extended_csp, crossword1, crossword2, crossword2d

p = Runtime_distribution(csp=extended_csp, xscale='log') # xscale is either 'linear' or 'log'
p.plot_run(num_runs=100, max_steps=1000, prob_best=0, prob_anycon=1.0)
p.plot_run(num_runs=100, max_steps=1000, prob_best=0.7, prob_anycon=1.0)